# Future Strategy - High Frequency Strategy 0001

## 1. Import Library

In [25]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [26]:
sd = '2025-11-01'
ed = '2025-11-15'

In [27]:
instrument_list = ['AP8888.CZC', 'PR8888.CZC', 'jm8888.DCE']

In [28]:
sql_strategy =  f"""
WITH
data_origin AS (
    SELECT
        date,
        instrument,
        m_avg(close, 20) AS ma_short,
        m_avg(close, 60) AS ma_long,
        IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_long,
        IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_short,
    FROM cn_future_bar1m
)
SELECT
    date,
    instrument,
    signal_long,
    signal_short
FROM data_origin
QUALIFY (signal_long = 1 OR signal_short = 1)
ORDER BY date, instrument
"""

In [29]:
df = dai.query(sql_strategy, filters={"date":[sd, ed], "instrument":instrument_list}).df()
df

,date,instrument,signal_long,signal_short
0,2025-11-03 10:04:00,PR8888.CZC,1,0
1,2025-11-03 10:10:00,jm8888.DCE,0,1
2,2025-11-03 10:32:00,AP8888.CZC,1,0
3,2025-11-03 10:49:00,PR8888.CZC,0,1
4,2025-11-03 13:32:00,AP8888.CZC,0,1
...,...,...,...,...
168,2025-11-14 21:55:00,jm8888.DCE,0,1
169,2025-11-14 22:00:00,PR8888.CZC,0,1
170,2025-11-14 22:19:00,jm8888.DCE,1,0
171,2025-11-14 22:38:00,jm8888.DCE,0,1


## 3. Backtest Strategy

In [30]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    context.subscribe_bar(context.instruments, '1m')

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d %H:%M:%S")]
    
    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction

    instruments_hold  = set(context.get_account_positions().keys())

    for ins in instruments_hold:
        position_long  = context.get_position(ins, Direction.LONG)
        position_short = context.get_position(ins, Direction.SHORT)
        price = data.current(ins,"open")
        if (position_long.current_qty  != 0):
            context.sell_close(ins, position_long.avail_qty, price, order_type=OrderType.MARKET)
        if (position_short.current_qty != 0):
            context.buy_close(ins, position_short.avail_qty, price, order_type=OrderType.MARKET)

    if len(df_now) == 0:
        return

    instruments_long  = set(df_now[df_now["signal_long"]  == 1]["instrument"])
    instruments_short = set(df_now[df_now["signal_short"] == 1]["instrument"])

    for ins in instruments_long:
        price = data.current(ins, "open")
        volume = 1
        context.buy_open(ins, volume, price, order_type=OrderType.MARKET)
    
    for ins in instruments_short:
        price = data.current(ins, "open")
        volume = 1
        context.sell_open(ins, volume, price, order_type=OrderType.MARKET)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 10000 + random.uniform(0, 10),
    frequency="""minute""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

BigTrader.raw_perf.read()

[2026-02-27 11:03:40] [info     ] bigtrader.v34 开始运行 ..
[2026-02-27 11:03:40] [info     ] 2025-11-03, 2025-11-14, , future, instruments=3
[2026-02-27 11:03:40] [info     ] bigtrader module V2.2.0
[2026-02-27 11:03:40] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10


ERROR:FUND[bktfut]:order_req freeze margin failed for symbol:AP8888.CZC, volume:1, order_price:8926.0, margin:7140.8


2026-02-27 11:03:41.441037 order_req freeze margin failed for symbol:AP8888.CZC, volume:1, order_price:8926.0, margin:7140.8 


ERROR:FUND[bktfut]:*update_frozen_detail 7137.6 not found frozen detail @symbol:AP8888.CZC,price:8926.0,volume:1,user_id:strategy


2026-02-27 11:03:41.442935 *update_frozen_detail 7137.6 not found frozen detail @symbol:AP8888.CZC,price:8926.0,volume:1,user_id:strategy 


...

2026-02-27 11:03:41.982350 order_req freeze margin failed for symbol:jm8888.DCE, volume:1, order_price:1217.0, margin:5841.6 


ERROR:FUND[bktfut]:*update_frozen_detail 5841.6 not found frozen detail @symbol:jm8888.DCE,price:1217.0,volume:1,user_id:strategy


2026-02-27 11:03:41.983933 *update_frozen_detail 5841.6 not found frozen detail @symbol:jm8888.DCE,price:1217.0,volume:1,user_id:strategy 
[2026-02-27 11:03:42] [info     ] backtest done, raw_perf_ds:dai.DataSource("_b8796406fac44005adf898674ef5c57c")


[2026-02-27 11:03:42] [info     ] bigtrader.v34 运行完成 [1.377s].


,date,trading_days,portfolio_value,starting_value,ending_value,starting_cash,ending_cash,returns,algorithm_period_return,benchmark_period_return,...,today_sell_balance,commission,capital_changed,orders,transactions,positions,POS_FAC,TRA_FAC,LOG,benchmark_returns
2025-11-03,2025-11-03,1,10223.988752,0.0,0.0,10005.988752,2994.788752,0.021787,0.021787,0.000000,...,63037.5,42.0,0.0,"[{'symbol': 'PR8888.CZC', 'id': '1', 'dt': 202...","[{'symbol': 'PR8888.CZC', 'dt': 2025-11-03 10:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[10:04:00][id:...",0.000000
2025-11-04,2025-11-04,2,10083.988752,0.0,0.0,2994.788752,10083.988752,-0.013693,0.007795,-0.007458,...,67010.5,50.0,0.0,"[{'symbol': 'AP8888.CZC', 'id': '23', 'dt': 20...","[{'symbol': 'AP8888.CZC', 'dt': 2025-11-04 09:...","[{'symbol': 'AP8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:01:00][id:...",-0.007458
2025-11-05,2025-11-05,3,10451.988752,0.0,0.0,10083.988752,10451.988752,0.036493,0.044573,-0.005619,...,68953.5,52.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '48', 'dt': 20...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-04 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:58:00][id:...",0.001853
2025-11-06,2025-11-06,4,10581.988752,0.0,0.0,10451.988752,10581.988752,0.012438,0.057566,0.008596,...,80491.0,60.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '74', 'dt': 20...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-05 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:24:00][id:...",0.014295
2025-11-07,2025-11-07,5,10639.988752,0.0,0.0,10581.988752,10639.988752,0.005481,0.063362,0.005457,...,115852.5,92.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '104', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-06 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:04:00][id:...",-0.003113
2025-11-10,2025-11-10,6,10359.988752,0.0,0.0,10639.988752,10359.988752,-0.026316,0.035379,0.008950,...,61451.0,60.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '150', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-07 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:17:00][id:...",0.003474
2025-11-11,2025-11-11,7,10117.988752,0.0,0.0,10359.988752,10117.988752,-0.023359,0.011193,-0.000266,...,92461.0,72.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '180', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-10 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:03:00][id:...",-0.009134
2025-11-12,2025-11-12,8,9753.988752,0.0,0.0,10117.988752,9753.988752,-0.035976,-0.025185,-0.001611,...,104017.0,84.0,0.0,"[{'symbol': 'PR8888.CZC', 'id': '216', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2025-11-11 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:16:00][id:...",-0.001345
2025-11-13,2025-11-13,9,10147.988752,0.0,0.0,9753.988752,10147.988752,0.040394,0.014192,0.010459,...,56273.0,56.0,0.0,"[{'symbol': 'PR8888.CZC', 'id': '258', 'dt': 2...","[{'symbol': 'PR8888.CZC', 'dt': 2025-11-12 21:...","[{'symbol': 'PR8888.CZC', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:06:00][id:...",0.012089
2025-11-14,2025-11-14,10,10223.988752,0.0,0.0,10147.988752,10223.988752,0.007489,0.021787,-0.005429,...,75101.5,64.0,0.0,"[{'symbol': 'jm8888.DCE', 'id': '286', 'dt': 2...","[{'symbol': 'jm8888.DCE', 'dt': 2025-11-13 21:...","[{'symbol': 'jm8888.DCE', 'direction': '1', 'c...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[21:08:00][id:...",-0.015724
